## Set up

In [1]:
%pip install tiktoken
%pip install transformers_stream_generator


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    %pip install circuitsvis
    
    # PySvelte is an unmaintained visualization library, use it as a backup if circuitsvis isn't working
    # # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Jupyter notebook - intended for development only!


/tmp/ipykernel_42473/410710250.py:21: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_42473/410710250.py:22: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [3]:
import torch
from transformers import AutoTokenizer
from transformer_lens import HookedEncoder
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

from transformer_lens import HookedTransformer

In [4]:
torch.set_grad_enabled(False)

## Load model

In [5]:
device = "cuda"
# device = "cpu"

fp16 = False
fp32 = True

In [6]:
model_path = "Qwen/Qwen-1_8B"
# model_path = "Qwen/Qwen-1_8B-Chat"
# model_path = "Qwen/Qwen-7B"
# model_path = "Qwen/Qwen-7B-Chat"
# model_path = "Qwen/Qwen-14B"
# model_path = "Qwen/Qwen-14B-Chat"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Local code to debug:

In [8]:
# from hf_qwen.modeling_qwen import QWenLMHeadModel
# model_hf = QWenLMHeadModel.from_pretrained(
#     model_path,
#     device_map=device,
#     fp32=True,
#     use_logn_attn=False,
#     use_dynamic_ntk = False,
#     scale_attn_weights = False,
# ).eval()

HF model:

In [9]:
model_hf = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map=device,
    trust_remote_code=True,
    fp16=fp16,
    fp32=fp32,
    use_logn_attn=False,
    use_dynamic_ntk = False,
    scale_attn_weights = False,
).eval()

Your device support faster inference by passing bf16=True in "AutoModelForCausalLM.from_pretrained".


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
model_tl = HookedTransformer.from_pretrained(
    model_path,
    fold_ln=False,
    trust_remote_code=True,
    center_writing_weights=False, 
    center_unembed=False,
    device=device,
    fp16=fp16,
    fp32=fp32,
)

Your device support faster inference by passing bf16=True in "AutoModelForCausalLM.from_pretrained".


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model Qwen/Qwen-1_8B into HookedTransformer


In [11]:
prompt = "This is a prompt to test if model_hf and model_tl are numerically close."
encoded_prompt = tokenizer.encode(prompt, return_tensors="pt").to(device)

In [12]:
result_hf = model_hf(encoded_prompt.to(device))
logits_hf = result_hf.logits

result_tl = model_tl(encoded_prompt.to(device))
logits_tl = result_tl

allclose_1 = torch.allclose(logits_hf, logits_tl, atol=1e-1)
allclose_2 = torch.allclose(logits_hf, logits_tl, atol=1e-2)
allclose_3 = torch.allclose(logits_hf, logits_tl, atol=1e-3)
allclose_4 = torch.allclose(logits_hf, logits_tl, atol=1e-4)
allclose_5 = torch.allclose(logits_hf, logits_tl, atol=1e-5)

print(f"allclose 1e-1: {allclose_1}")
print(f"allclose 1e-2: {allclose_2}")
print(f"allclose 1e-3: {allclose_3}")
print(f"allclose 1e-4: {allclose_4}")
print(f"allclose 1e-5: {allclose_5}")

allclose 1e-1: True
allclose 1e-2: True
allclose 1e-3: True
allclose 1e-4: True
allclose 1e-5: False
